In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import datetime as datetime
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("BPI Challenge 2017.csv", parse_dates = ['time:timestamp'])
df.info()
# The default name indicating the case ID is case:concept:name
# concept:name is the event
# time:timestamp is the corresponding timestamp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202267 entries, 0 to 1202266
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype              
---  ------                 --------------    -----              
 0   Unnamed: 0             1202267 non-null  int64              
 1   Action                 1202267 non-null  object             
 2   org:resource           1202267 non-null  object             
 3   concept:name           1202267 non-null  object             
 4   EventOrigin            1202267 non-null  object             
 5   EventID                1202267 non-null  object             
 6   lifecycle:transition   1202267 non-null  object             
 7   time:timestamp         1202267 non-null  datetime64[ns, UTC]
 8   case:LoanGoal          1202267 non-null  object             
 9   case:ApplicationType   1202267 non-null  object             
 10  case:concept:name      1202267 non-null  object             
 11  case:RequestedAmount   1

# 1. Splitting Data

In [3]:
# Obtain date (datetime format) from datatype of time:timestamp 
df['Date'] = np.array(df['time:timestamp'].values, dtype = 'datetime64[D]').astype(datetime.datetime)
df

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,Date
0,0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01
1,1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01
2,2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,2017-01-06 06:33:02.212000+00:00,Home improvement,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-06
1202263,1202263,Created,User_1,W_Call after offers,Workflow,Workitem_363876066,schedule,2017-01-06 06:33:02.221000+00:00,Home improvement,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-06
1202264,1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114000+00:00,Home improvement,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-16
1202265,1202265,statechange,User_28,O_Cancelled,Offer,OfferState_420066181,complete,2017-01-16 09:51:21.139000+00:00,Home improvement,New credit,...,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1580299144,2017-01-16


In [4]:
# Determine training and testing data's date boundaries
date_unique = sorted(df['Date'].unique())
total_date = len(date_unique)
all_train_nr = round(total_date * 0.8)
date_before_test = date_unique[all_train_nr - 1]
date_before_test

datetime.date(2016, 11, 13)

In [5]:
# Remove entries with case ID across date boundaries
small_df = df[['Date', 'case:concept:name']].drop_duplicates()
small_df_1 = small_df[small_df['Date'] <= date_before_test]
small_df_2 = small_df[small_df['Date'] > date_before_test]
bruh = set(small_df_1['case:concept:name'].unique()).intersection(set(small_df_2['case:concept:name'].unique()))
case_unique_train = sorted(list(set(small_df_1['case:concept:name'].unique()) - bruh))
case_unique_test = sorted(list(set(small_df_2['case:concept:name'].unique()) - bruh))

In [6]:
# Determine training and testing data's ID boundaries
all_case = sorted(df['case:concept:name'].unique())
total_case = len(all_case)
all_train_case = round(total_case * 0.8)
case_all_train = sorted(all_case)[: all_train_case]
case_test = sorted(all_case)[all_train_case: ]

# Combine ID boundaries and time boundaries
final_all_train = sorted(list(set(case_unique_train).intersection(set(case_all_train))))
final_test = sorted(list(set(case_unique_test).intersection(set(case_test))))

# Split training and validation dataset
final_train, final_val = train_test_split(final_all_train, test_size = 0.2)

# Split the dataset
df_train = df[df['case:concept:name'].isin(final_train)]
df_val = df[df['case:concept:name'].isin(final_val)]
df_test = df[df['case:concept:name'].isin(final_test)]
df_train = df_train.drop(columns = ['Unnamed: 0', 'Date']).reset_index(drop = True)
df_val = df_val.drop(columns = ['Unnamed: 0', 'Date']).reset_index(drop = True)
df_test = df_test.drop(columns = ['Unnamed: 0', 'Date']).reset_index(drop = True)

In [7]:
df_train

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621126,Obtained,User_43,W_Call after offers,Workflow,Workitem_162257588,start,2016-11-12 09:31:06.232000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621127,statechange,User_43,A_Complete,Application,ApplState_1435859997,complete,2016-11-12 09:31:06.234000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621128,statechange,User_43,A_Cancelled,Application,ApplState_860885554,complete,2016-11-12 09:35:55.142000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621129,statechange,User_43,O_Cancelled,Offer,OfferState_2011904028,complete,2016-11-12 09:35:55.161000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1248413119


In [8]:
df_val

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_1746793196,complete,2016-01-01 12:34:53.911000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_282472804,complete,2016-01-01 12:34:53.950000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1676837235,schedule,2016-01-01 12:34:54.320000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Obtained,User_17,W_Handle leads,Workflow,Workitem_1237205341,start,2016-01-02 09:05:02.658000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Deleted,User_17,W_Handle leads,Workflow,Workitem_548358991,complete,2016-01-02 09:06:04.201000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154996,Created,User_7,O_Create Offer,Offer,Offer_915670841,complete,2016-11-10 12:32:01.087000+00:00,Home improvement,New credit,Application_1929025400,15000.0,0.0,47.0,False,350.0,False,0.0,15000.0,NaN
154997,statechange,User_7,O_Created,Offer,OfferState_377748319,complete,2016-11-10 12:32:01.672000+00:00,Home improvement,New credit,Application_1929025400,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_915670841
154998,statechange,User_7,A_Cancelled,Application,ApplState_1946614717,complete,2016-11-10 12:38:03.730000+00:00,Home improvement,New credit,Application_1929025400,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154999,statechange,User_7,O_Cancelled,Offer,OfferState_942667715,complete,2016-11-10 12:38:03.746000+00:00,Home improvement,New credit,Application_1929025400,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_915670841


In [9]:
df_test

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_837911105,complete,2016-11-14 05:57:57.461000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1692094008,complete,2016-11-14 05:57:59.458000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1100633268,schedule,2016-11-14 05:57:59.981000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Obtained,User_54,W_Handle leads,Workflow,Workitem_167603855,start,2016-11-14 08:27:08.979000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Deleted,User_54,W_Handle leads,Workflow,Workitem_1887085579,complete,2016-11-14 08:27:59.637000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28701,Obtained,User_49,W_Call incomplete files,Workflow,Workitem_1264169641,resume,2017-01-06 16:13:55.507000+00:00,Car,New credit,Application_965278193,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28702,Released,User_49,W_Call incomplete files,Workflow,Workitem_2138456013,suspend,2017-01-06 16:16:17.741000+00:00,Car,New credit,Application_965278193,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28703,statechange,User_133,O_Accepted,Offer,OfferState_224690953,complete,2017-01-10 15:35:46.239000+00:00,Car,New credit,Application_965278193,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1880706415
28704,statechange,User_133,A_Pending,Application,ApplState_2038793609,complete,2017-01-10 15:35:46.242000+00:00,Car,New credit,Application_965278193,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_train.to_csv('bpi2017_train.csv')
df_val.to_csv('bpi2017_val.csv')
df_test.to_csv('bpi2017_test.csv')

# 2. Baseline Time Prediction (Only on Training Dataset)

In [11]:
# Calculate time difference
df_train['time_diff'] = df_train['time:timestamp'].diff().dt.total_seconds()
# Set the time difference of the 1st row 0
df_train.loc[0, 'time_diff'] = 0
# Count number of processes per trace/ID
count_lst = df_train.groupby('case:concept:name').count()['time_diff'].tolist()
# Assign position number to each row/process
position_lst_1 = [list(range(1, i + 1)) for i in count_lst]
position_lst = []
for i in position_lst_1:
    for j in i:
        position_lst.append(j)
df_train['position'] = position_lst
# Set the time difference of every process with position = 1 as 0
df_train.loc[df_train['position'] == 1, 'time_diff'] = 0
df_train

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,time_diff,position
0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1
1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049,2
2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.191,3
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.833,4
4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621126,Obtained,User_43,W_Call after offers,Workflow,Workitem_162257588,start,2016-11-12 09:31:06.232000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002,67
621127,statechange,User_43,A_Complete,Application,ApplState_1435859997,complete,2016-11-12 09:31:06.234000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002,68
621128,statechange,User_43,A_Cancelled,Application,ApplState_860885554,complete,2016-11-12 09:35:55.142000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288.908,69
621129,statechange,User_43,O_Cancelled,Offer,OfferState_2011904028,complete,2016-11-12 09:35:55.161000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1248413119,0.019,70


In [12]:
# Calculate mean time difference grouped by position based on the number of cases
mean_time_lst = df_train.groupby('position').mean()['time_diff'].tolist()
# Delete 1st time difference mean (Position = 1 always has mean = 0)
del mean_time_lst[0]
# Add the last time differnce mean as 0 when it reaches the maximum value of position
mean_time_lst.append(0.0)
# Create the predicted time column per entry using the mean time difference
pred_time_lst = [mean_time_lst[j - 1] for j in position_lst]
df_train['baseline_predicted_time'] = pred_time_lst
df_train

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,time_diff,position,baseline_predicted_time
0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1,-7277.466942
1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049,2,-2010.479743
2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.191,3,1758.541528
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.833,4,3209.420304
4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,Home improvement,New credit,Application_1691306052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011,5,-1153.974426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621126,Obtained,User_43,W_Call after offers,Workflow,Workitem_162257588,start,2016-11-12 09:31:06.232000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002,67,-2516.572518
621127,statechange,User_43,A_Complete,Application,ApplState_1435859997,complete,2016-11-12 09:31:06.234000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002,68,-9721.636303
621128,statechange,User_43,A_Cancelled,Application,ApplState_860885554,complete,2016-11-12 09:35:55.142000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288.908,69,5422.906190
621129,statechange,User_43,O_Cancelled,Offer,OfferState_2011904028,complete,2016-11-12 09:35:55.161000+00:00,Not speficied,New credit,Application_1583440896,...,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1248413119,0.019,70,39800.612477


# 3. Apply Above Calculated Mean Time to Validation and Test Set

In [13]:
# Calculate time difference
df_val['time_diff'] = df_val['time:timestamp'].diff().dt.total_seconds()
# Set the time difference of the 1st row 0
df_val.loc[0, 'time_diff'] = 0
# Count number of processes per trace/ID
count_val_lst = df_val.groupby('case:concept:name').count()['time_diff'].tolist()
# Assign position number to each row/process
position_lst_1_val = [list(range(1, i + 1)) for i in count_val_lst]
position_lst_val = []
for i in position_lst_1_val:
    for j in i:
        position_lst_val.append(j)
df_val['position'] = position_lst_val
# Set the time difference of every process with position = 1 as 0
df_val.loc[df_val['position'] == 1, 'time_diff'] = 0
# Create the predicted time column per entry using the mean time difference
pred_time_lst_val = [mean_time_lst[j - 1] for j in position_lst_val]
df_val['baseline_predicted_time'] = pred_time_lst_val
df_val

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,time_diff,position,baseline_predicted_time
0,Created,User_1,A_Create Application,Application,Application_1746793196,complete,2016-01-01 12:34:53.911000+00:00,Car,New credit,Application_1746793196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1,-7277.466942
1,statechange,User_1,A_Submitted,Application,ApplState_282472804,complete,2016-01-01 12:34:53.950000+00:00,Car,New credit,Application_1746793196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039,2,-2010.479743
2,Created,User_1,W_Handle leads,Workflow,Workitem_1676837235,schedule,2016-01-01 12:34:54.320000+00:00,Car,New credit,Application_1746793196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.370,3,1758.541528
3,Obtained,User_17,W_Handle leads,Workflow,Workitem_1237205341,start,2016-01-02 09:05:02.658000+00:00,Car,New credit,Application_1746793196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73808.338,4,3209.420304
4,Deleted,User_17,W_Handle leads,Workflow,Workitem_548358991,complete,2016-01-02 09:06:04.201000+00:00,Car,New credit,Application_1746793196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.543,5,-1153.974426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154996,Created,User_7,O_Create Offer,Offer,Offer_915670841,complete,2016-11-10 12:32:01.087000+00:00,Home improvement,New credit,Application_1929025400,...,47.0,False,350.0,False,0.0,15000.0,NaN,399.986,56,-21495.840666
154997,statechange,User_7,O_Created,Offer,OfferState_377748319,complete,2016-11-10 12:32:01.672000+00:00,Home improvement,New credit,Application_1929025400,...,NaN,NaN,NaN,NaN,NaN,NaN,Offer_915670841,0.585,57,11480.656083
154998,statechange,User_7,A_Cancelled,Application,ApplState_1946614717,complete,2016-11-10 12:38:03.730000+00:00,Home improvement,New credit,Application_1929025400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,362.058,58,-553.326632
154999,statechange,User_7,O_Cancelled,Offer,OfferState_942667715,complete,2016-11-10 12:38:03.746000+00:00,Home improvement,New credit,Application_1929025400,...,NaN,NaN,NaN,NaN,NaN,NaN,Offer_915670841,0.016,59,3501.404450


In [14]:
# Calculate time difference
df_test['time_diff'] = df_test['time:timestamp'].diff().dt.total_seconds()
# Set the time difference of the 1st row 0
df_test.loc[0, 'time_diff'] = 0
# Count number of processes per trace/ID
count_test_lst = df_test.groupby('case:concept:name').count()['time_diff'].tolist()
# Assign position number to each row/process
position_lst_1_test = [list(range(1, i + 1)) for i in count_test_lst]
position_lst_test = []
for i in position_lst_1_test:
    for j in i:
        position_lst_test.append(j)
df_test['position'] = position_lst_test
# Set the time difference of every process with position = 1 as 0
df_test.loc[df_test['position'] == 1, 'time_diff'] = 0
# Create the predicted time column per entry using the mean time difference
pred_time_lst_test = [mean_time_lst[j - 1] for j in position_lst_test]
df_test['baseline_predicted_time'] = pred_time_lst_test
df_test

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,time_diff,position,baseline_predicted_time
0,Created,User_1,A_Create Application,Application,Application_837911105,complete,2016-11-14 05:57:57.461000+00:00,Existing loan takeover,New credit,Application_837911105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1,-7277.466942
1,statechange,User_1,A_Submitted,Application,ApplState_1692094008,complete,2016-11-14 05:57:59.458000+00:00,Existing loan takeover,New credit,Application_837911105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.997,2,-2010.479743
2,Created,User_1,W_Handle leads,Workflow,Workitem_1100633268,schedule,2016-11-14 05:57:59.981000+00:00,Existing loan takeover,New credit,Application_837911105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.523,3,1758.541528
3,Obtained,User_54,W_Handle leads,Workflow,Workitem_167603855,start,2016-11-14 08:27:08.979000+00:00,Existing loan takeover,New credit,Application_837911105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8948.998,4,3209.420304
4,Deleted,User_54,W_Handle leads,Workflow,Workitem_1887085579,complete,2016-11-14 08:27:59.637000+00:00,Existing loan takeover,New credit,Application_837911105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.658,5,-1153.974426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28701,Obtained,User_49,W_Call incomplete files,Workflow,Workitem_1264169641,resume,2017-01-06 16:13:55.507000+00:00,Car,New credit,Application_965278193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6092.978,19,-4689.105398
28702,Released,User_49,W_Call incomplete files,Workflow,Workitem_2138456013,suspend,2017-01-06 16:16:17.741000+00:00,Car,New credit,Application_965278193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142.234,20,1448.663456
28703,statechange,User_133,O_Accepted,Offer,OfferState_224690953,complete,2017-01-10 15:35:46.239000+00:00,Car,New credit,Application_965278193,...,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1880706415,343168.498,21,953.777303
28704,statechange,User_133,A_Pending,Application,ApplState_2038793609,complete,2017-01-10 15:35:46.242000+00:00,Car,New credit,Application_965278193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003,22,6065.267463
